In [2]:
################################ EDA IMPORTS ###################################
%matplotlib inline
import pandas as pd 
import pandas_profiling # Quick Data Analysis
import numpy as np # Linear Algebra lib
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.graph_objs as go # interactive low-level plotting lib https://plot.ly/python/
import plotly.express as px #high-level api wrapper for plotly https://plot.ly/python/plotly-express/#visualize-distributions
# ---------------- Plot libs settings ------------- #
# Pick style of Matplolib plots 
# Different style sheets:-> https://matplotlib.org/3.1.0/gallery/style_sheets/style_sheets_reference.html
# Configure Seaborn Asthetics: -> https://seaborn.pydata.org/tutorial/aesthetics.html?highlight=style
plt.style.use('seaborn-darkgrid')
sns.set(context='notebook', style='darkgrid', palette='colorblind')
# Seting a universal figure size 
plt.rcParams['figure.figsize'] = (10, 6)

# ---------------- Pandas settings ------------- #
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200

################################################################################

#################################### ML IMPORTS ################################
### Pre-Processing
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

### Train/Test - Split & CV
from sklearn.model_selection import train_test_split

### Regressors
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

### Pipeline
from sklearn.pipeline import make_pipeline

### Feature Importance
import eli5
from eli5.sklearn import PermutationImportance

### Tuning Hyperparameters
from sklearn.model_selection import GridSearchCV



################################################################################

# Project PUBG 🤖💣💥🔫🤖 

![](https://o.aolcdn.com/images/dims?quality=100&image_uri=http%3A%2F%2Fo.aolcdn.com%2Fhss%2Fstorage%2Fmidas%2Fb0be09f425cc5175fb413bc03c32dd0d%2F206235889%2Fpubg-ed.jpg&client=amp-blogside-v2&signature=88c6b77342cbeb0d25c0dc9d909018136aec1971)

### Load Dataset

In [4]:
def load_dataset(f, verbose=True, develop=True):
    df = pd.read_csv('../pubg-api/pubg_stats.csv')
    df = reduce_mem_usage(df, verbose=verbose)
    if verbose:
        print('-' * 80)
        print(
        f'''
        -------------------- SHAPE ---------------------
        PUBG Stats: {df.shape}
        ------------------------------------------------
        ''')
    return df

train = load_dataset('../clean-datasets/train.csv')
val = load_dataset('../clean-datasets/validate.csv')
test = load_dataset('../clean-datasets/test.csv')

train.head()

Memory usage of dataframe is 17.83 MB
Memory usage after optimization is: 6.69 MB
Decreased by 62.5%
--------------------------------------------------------------------------------
Load: train

        -------------------- SHAPE ---------------------
        PUBG Stats: (83480, 28)
        ------------------------------------------------
        


,DBNOs,assists,boosts,damageDealt,deathType,duration,gameMode,headshotKills,heals,killPlace,killStreaks,kills,longestKill,mapName,matchId,name,platform,playerId,revives,rideDistance,roadKills,swimDistance,teamKills,timeSurvived,vehicleDestroys,walkDistance,weaponsAcquired,winPlace
0,1,2,0,118.5625,byplayer,1787,squad,0,0,35,1,1,8.742188,Baltic_Main,495f2895-a0a2-41a3-9faa-e64056f6364c,CoreyMrT,steam,account.e8dec15d50f44b33b5ecbd77b4498efe,0,0.0,0,0.0,0,375.25,0,417.75,4,22
1,0,0,5,166.0000,byplayer,1787,squad,0,1,11,1,2,120.562500,Baltic_Main,495f2895-a0a2-41a3-9faa-e64056f6364c,MichaelTX254,steam,account.83e9e5bd6566499da1cc5f185c43eb74,1,3156.0,0,0.0,0,1421.00,0,2608.00,5,5
2,2,1,4,266.0000,byplayer,1787,squad,0,5,7,2,3,23.062500,Baltic_Main,495f2895-a0a2-41a3-9faa-e64056f6364c,Dr_Matasanos,steam,account.97dc2011adfe4c75879743f83903fb59,0,2846.0,0,0.0,0,1400.00,0,3002.00,4,6
3,1,5,3,417.2500,byzone,1787,squad,0,0,5,2,3,270.500000,Baltic_Main,495f2895-a0a2-41a3-9faa-e64056f6364c,Pray_GM,steam,account.3d478dcf49f449b3a42e84063a621074,0,1302.0,0,0.0,0,1549.00,1,2676.00,7,1
4,1,0,0,18.0000,byplayer,1787,squad,0,0,84,0,0,0.000000,Baltic_Main,495f2895-a0a2-41a3-9faa-e64056f6364c,ComicBooksPlus,steam,account.534f2b758b01409681fe3c4313ad270a,0,0.0,0,0.0,0,301.50,0,530.00,3,24


## Feature Engineering

In [ ]:
mask = (train['duration'] < 800) & (train['walkDistance'] == 0)
train[mask]

winPlace == 1 and walkDistance == 0 are cheaters or hacks in glitches

drop outliers:

- high weapons picked up
- high ride distance, no kills
- high walkdistance, no kills
- high heals, no kills


drop team killers



In [ ]:
def feature_engineering(df):
    
    # Remove players that won with no kills, or winPlacePerc > 0.9
    
    
    
    
    df['headKillsRatio'] = df['headshotKills'] / df['kills']
    




## Random Forest

## Light GBM